<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=acf62a160619aefdd455237f37f7406908d7fd07d45d97ff08a571ba3b7da786
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-23 09:26:19
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: -19.82 K (-0.14%)
Current PnL: -25.95 L (-16.76%)
CY Booked + Current PnL: -11.66 L (-7.53%)
-------------------
Total profit:  1.55 L
Total loss:  -27.49 L
-------------------
Total Booked + Current PnL: 15.32 L (11.99%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.13%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.58 L (64.19%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.45%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.75,-18.63,23.01,0.09,18868.0,-18778.0,81999.0,90.39,40.0,M-SC,2.38,253.0,-1.00,0.58,7.73,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.35,8.21,10.61,19.69,22014.0,15738.0,207480.0,-2.32,53.0,X-MC,2.47,78.0,0.71,1.48,15.56,XY25,NTT,AC
50,MASFIN,398.61,0.36,-2.91,25.70,22.05,24448.0,-2850.0,95130.0,-16.44,52.0,H-SC,6.29,164.0,-0.12,0.68,38.28,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-0.02,-16.18,19.32,0.02,28269.0,-28239.0,146320.0,123.21,42.0,M-SC,12.06,234.0,-1.00,1.04,27.27,OX40N,NTT,PAINTS
43,ITC,452.00,-0.03,-1.33,12.27,10.78,29096.0,-3187.0,237131.0,-41.15,49.0,X-LC,2.13,5.0,-0.11,1.69,3.84,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,2.21,-2.91,22.80,19.23,33305.0,-4384.0,146076.0,14.04,69.0,L-LC,2.05,179.0,-0.13,1.04,17.74,XY25,ATH,MINING
79,VAIBHAVGBL,521.00,1.96,-26.10,117.54,60.77,158771.0,-47697.0,135078.0,55.92,50.0,H-SC,4.69,159.0,-0.30,0.96,21.65,XR,NTT,JEWELLERY
60,REPCOHOME,880.00,1.68,-13.72,120.00,89.82,296640.0,-39305.0,247200.0,-56.78,49.0,H-SC,5.59,160.0,-0.13,1.76,30.79,XY24,NTT,FINANCE
78,UNITDSPR,1644.00,1.42,4.80,15.23,20.77,37373.0,11248.0,245392.0,-4.29,50.0,X-MC,2.18,62.0,0.30,1.75,12.05,X40N,NTT,BREWERIES
80,VALIANTORG,838.00,1.16,-44.00,211.00,74.15,193303.0,-71992.0,91613.0,-264.69,48.0,H-SC,22.51,149.0,-0.37,0.65,17.43,XR,NTT,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,-3.20,-56.87,132.00,0.07,78103.0,-78008.0,59169.0,2069.19,71.0,L-SC,3.63,268.0,-1.00,0.42,47.16,OX40N,NTT,JEWELLERY
41,INFY,1972.00,-1.94,12.79,19.02,34.24,67121.0,40010.0,352898.0,-12.06,65.0,X-LC,5.77,2.0,0.60,2.51,22.13,X40,NTT,IT
55,QUESS,424.00,-1.48,-29.56,102.77,42.84,47061.0,-19213.0,45793.0,-53.72,45.0,X-SC,34.27,83.0,-0.41,0.33,3.60,XY24,NTT,MISC
22,DIXON,18931.72,-1.46,-15.72,49.58,26.07,81579.0,-30680.0,164541.0,-59.02,31.0,X-MC,11.91,56.0,-0.38,1.17,2.48,X40N,ATH,IT
67,SONACOMS,806.63,-1.37,-15.80,65.73,39.54,55984.0,-15987.0,85172.0,-34.10,48.0,M-SC,3.85,220.0,-0.29,0.61,20.10,AR,ATH,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.56,-1.18,110.40,107.91,161070.0,-1747.0,145897.0,-19.74,57.0,M-SC,11.89,216.0,-0.01,1.04,8.78,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.02,-16.18,19.32,0.02,28269.0,-28239.0,146320.0,123.21,42.0,M-SC,12.06,234.0,-1.00,1.04,27.27,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.75,-18.63,23.01,0.09,18868.0,-18778.0,81999.0,90.39,40.0,M-SC,2.38,253.0,-1.00,0.58,7.73,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,-0.71,-21.89,69.04,32.03,71090.0,-28865.0,102970.0,-15.53,61.0,H-SC,9.11,148.0,-0.41,0.73,15.34,OX40N,NTT,IT
66,SIS,528.00,-0.41,-23.81,59.16,21.26,50047.0,-26436.0,84596.0,1996.37,52.0,H-SC,4.71,166.0,-0.53,0.60,14.30,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.61,11.15,54.98,72.27,103552.0,18897.0,188345.0,-4.72,73.0,M-LC,3.68,99.0,0.18,1.34,18.02,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.61,11.15,54.98,72.27,103552.0,18897.0,188345.0,-4.72,73.0,M-LC,3.68,99.0,0.18,1.34,18.02,XR,NTT,IT
25,FINCABLES,1641.55,-0.56,-1.18,110.40,107.91,161070.0,-1747.0,145897.0,-19.74,57.0,M-SC,11.89,216.0,-0.01,1.04,8.78,OX40N,ATH,CABLES
6,ASIANTILES,137.00,-0.58,-2.76,86.83,81.67,79209.0,-2587.0,91223.0,8216.67,72.0,L-SC,12.25,271.0,-0.03,0.65,76.66,XR,NTT,CERAMICS
38,INDIAMART,4810.62,0.15,-4.54,116.55,106.72,137226.0,-5596.0,117740.0,-52.73,37.0,H-SC,4.28,139.0,-0.04,0.84,17.78,AR,ATH,MISC
37,IEX,219.00,0.18,-4.17,55.10,48.63,105576.0,-8346.0,191608.0,-36.24,49.0,H-SC,13.47,136.0,-0.08,1.36,6.90,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-1.46,-15.72,49.58,26.07,81579.0,-30680.0,164541.0,-59.02,31.0,X-MC,11.91,56.0,-0.38,1.17,2.48,X40N,ATH,IT
11,BATAINDIA,2096.00,-0.04,-38.87,121.17,35.20,95311.0,-50011.0,78659.0,3.10,34.0,X-SC,16.87,92.0,-0.52,0.56,0.89,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,0.29,-17.61,88.71,55.47,177809.0,-42847.0,200439.0,-26.96,34.0,X-MC,15.18,64.0,-0.24,1.43,2.70,X40N,ATH,FINANCE
8,AWL,485.00,0.08,-23.44,100.50,53.50,232169.0,-70737.0,231014.0,-62.99,35.0,X-MC,5.37,58.0,-0.30,1.64,2.61,XY24,NTT,FMCG
20,DABUR,735.00,-0.16,-3.63,49.12,43.70,117424.0,-9012.0,239056.0,-11.62,35.0,X-MC,2.08,72.0,-0.08,1.70,10.87,XY24,BTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.18,-0.31,24.87,24.48,62440.0,-792.0,251064.0,-81.09,49.0,X-SC,1.17,86.0,-0.01,1.79,22.75,X40N,NTT,MISC
12,BERGEPAINT,680.0,0.90,-2.54,24.30,21.14,53838.0,-5784.0,221555.0,-9.49,48.0,X-MC,1.18,74.0,-0.11,1.58,24.88,XY24,NTT,PAINTS
20,DABUR,735.0,-0.16,-3.63,49.12,43.70,117424.0,-9012.0,239056.0,-11.62,35.0,X-MC,2.08,72.0,-0.08,1.70,10.87,XY24,BTT,FMCG
43,ITC,452.0,-0.03,-1.33,12.27,10.78,29096.0,-3187.0,237131.0,-41.15,49.0,X-LC,2.13,5.0,-0.11,1.69,3.84,X40,NTT,FMCG
78,UNITDSPR,1644.0,1.42,4.80,15.23,20.77,37373.0,11248.0,245392.0,-4.29,50.0,X-MC,2.18,62.0,0.30,1.75,12.05,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COLPAL,3726.84,-0.43,-20.30,77.54,41.51,162764.0,-53455.0,209910.0,-4.98,38.0,X-MC,7.75,61.0,-0.33,1.49,0.56,XY25,ATH,FMCG
3,ACC,3906.00,-0.99,-25.83,121.33,64.15,214123.0,-61471.0,176480.0,-56.75,38.0,X-SC,3.80,82.0,-0.29,1.26,0.76,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.04,-38.87,121.17,35.20,95311.0,-50011.0,78659.0,3.10,34.0,X-SC,16.87,92.0,-0.52,0.56,0.89,X40,NTT,FOOTWEAR
59,RELAXO,1176.00,0.12,-47.82,191.88,52.31,145340.0,-69415.0,75745.0,-44.44,42.0,X-SC,7.27,91.0,-0.48,0.54,1.39,X40N,NTT,FOOTWEAR
22,DIXON,18931.72,-1.46,-15.72,49.58,26.07,81579.0,-30680.0,164541.0,-59.02,31.0,X-MC,11.91,56.0,-0.38,1.17,2.48,X40N,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.48,-29.56,102.77,42.84,47061.0,-19213.0,45793.0,-53.72,45.0,X-SC,34.27,83.0,-0.41,0.33,3.60,XY24,NTT,MISC
59,RELAXO,1176.00,0.12,-47.82,191.88,52.31,145340.0,-69415.0,75745.0,-44.44,42.0,X-SC,7.27,91.0,-0.48,0.54,1.39,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.04,-38.87,121.17,35.20,95311.0,-50011.0,78659.0,3.10,34.0,X-SC,16.87,92.0,-0.52,0.56,0.89,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,0.05,-2.41,27.14,24.08,33625.0,-3057.0,123893.0,-10.30,42.0,X-SC,6.18,89.0,-0.09,0.88,16.74,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.27,-17.26,73.22,43.32,98671.0,-28112.0,134760.0,-28.92,36.0,X-SC,5.61,90.0,-0.28,0.96,3.47,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.81,-9.32,33.11,20.71,113565.0,-35245.0,342992.0,-22.01,67.0,X-LC,3.47,1.0,-0.31,2.44,14.61,X40,ATH,IT
41,INFY,1972.00,-1.94,12.79,19.02,34.24,67121.0,40010.0,352898.0,-12.06,65.0,X-LC,5.77,2.0,0.60,2.51,22.13,X40,NTT,IT
75,TMPV,600.00,0.10,-26.32,66.90,22.98,111354.0,-59450.0,166448.0,-23.08,50.0,X-LC,6.46,3.0,-0.53,1.19,4.69,XY24,NTT,AUTO
81,VBL,671.64,-0.79,-2.68,39.40,35.67,121111.0,-8454.0,307388.0,-14.21,57.0,X-LC,3.61,4.0,-0.07,2.19,10.81,X40N,ATH,FMCG
43,ITC,452.00,-0.03,-1.33,12.27,10.78,29096.0,-3187.0,237131.0,-41.15,49.0,X-LC,2.13,5.0,-0.11,1.69,3.84,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.10,-36.71,109.96,32.89,53733.0,-28340.0,48866.0,-703.29,58.0,L-MC,6.86,259.0,-0.53,0.35,34.57,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-3.20,-56.87,132.00,0.07,78103.0,-78008.0,59169.0,2069.19,71.0,L-SC,3.63,268.0,-1.00,0.42,47.16,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-0.58,-2.76,86.83,81.67,79209.0,-2587.0,91223.0,8216.67,72.0,L-SC,12.25,271.0,-0.03,0.65,76.66,XR,NTT,CERAMICS
50,MASFIN,398.61,0.36,-2.91,25.70,22.05,24448.0,-2850.0,95130.0,-16.44,52.0,H-SC,6.29,164.0,-0.12,0.68,38.28,XR,ATH,FINANCE
13,BSOFT,831.70,-0.50,-18.80,85.69,50.79,97870.0,-26439.0,114214.0,3.98,70.0,H-SC,7.58,151.0,-0.27,0.81,32.98,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.50,-18.80,85.69,50.79,97870.0,-26439.0,114214.0,3.98,70.0,H-SC,7.58,151.0,-0.27,0.81,32.98,XR,ATH,IT
56,RAJESHEXPO,518.00,-3.20,-56.87,132.00,0.07,78103.0,-78008.0,59169.0,2069.19,71.0,L-SC,3.63,268.0,-1.00,0.42,47.16,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-0.58,-2.76,86.83,81.67,79209.0,-2587.0,91223.0,8216.67,72.0,L-SC,12.25,271.0,-0.03,0.65,76.66,XR,NTT,CERAMICS
61,ROUTE,2227.21,-0.67,-46.00,213.03,69.05,153088.0,-61204.0,71862.0,-56.18,54.0,H-SC,21.01,140.0,-0.40,0.51,10.83,SR,ATH,IT
84,WIPRO,420.00,-0.61,11.15,54.98,72.27,103552.0,18897.0,188345.0,-4.72,73.0,M-LC,3.68,99.0,0.18,1.34,18.02,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.38
1,25,44.83
2,50,76.55


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.96
MC    30.76
LC    24.28
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.80
X40      23.03
X40N     12.96
XR        9.89
XY25      9.08
AR        8.91
OX40N     7.58
SR        1.00
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.26
H-SC    23.75
X-LC    20.79
M-SC    11.64
X-SC     8.03
H-MC     4.79
L-SC     1.54
M-MC     1.36
M-LC     1.34
H-LC     1.11
L-LC     1.04
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.24,-7.70,42.34
IT,13.11,-14.05,71.39
FINANCE,9.53,-19.15,70.96
MISC,7.28,-27.20,79.24
ELECTRICAL,5.97,-11.98,52.99
PAINTS,5.80,-13.30,30.13
INSURANCE,4.76,-1.48,36.21
PHARMA,4.06,-3.06,35.50
AUTO,2.86,-30.93,76.33


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3158871.0,21
AR,1322542.0,10
XR,1272871.0,13
X40,988361.0,14
X40N,865651.0,9
OX40N,725732.0,10
XY25,366196.0,6
SR,279520.0,2
MH,78325.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3642292.0,25
M-SC,1424478.0,15
X-MC,1418984.0,16
X-LC,806018.0,11
X-SC,792493.0,8
H-MC,406051.0,3
L-SC,244462.0,3
M-LC,103552.0,1
H-LC,77765.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1270237.0      6
           AR         905169.0      5
M-SC       XY24       811800.0      6
H-SC       XR         776163.0      7
X-MC       X40        487121.0      7
           XY24       403431.0      3
           X40N       343654.0      4
H-SC       OX40N      332878.0      4
M-SC       OX40N      314751.0      5
X-LC       X40        310007.0      5
X-SC       X40N       306451.0      3
           XY24       294809.0      3
H-SC       SR         279520.0      2
X-LC       X40N       215546.0      2
H-MC       AR         214745.0      2
           XY24       191306.0      1
X-SC       X40        191233.0      2
X-LC       XY24       187288.0      2
X-MC       XY25       184778.0      2
M-SC       XR         173064.0      2
L-SC       XR         166359.0      2
M-SC       AR         124863.0      2
M-LC       XR         103552.0      1
X-LC       XY25        93177.0      2
H-SC       MH          78325.0      1
L-SC       OX40N       78103.0      1
H-LC       AR          77765.0      1
M-MC       XY25        54936.0      1
L-MC       XR          53733.0      1
L-LC       XY25        33305.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 57.0 seconds
